In [1]:
import langchain
from dotenv import load_dotenv
import os

In [4]:
load_dotenv()  # This loads the .env variables into the environment

secret_key = os.getenv('SQL_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')